All the codes are rearranged from controlnet annotator

## Import libary

In [23]:
from openpose import OpenposeDetector
from PIL import Image
import cv2
import numpy as np
import torch

In [17]:
image_resolution = 512
detect_resolution = 512
images_path = ["motion_frames/Bandit"]

In [18]:
def HWC3(x):
    assert x.dtype == np.uint8
    if x.ndim == 2:
        x = x[:, :, None]
    assert x.ndim == 3
    H, W, C = x.shape
    assert C == 1 or C == 3 or C == 4
    if C == 3:
        return x
    if C == 1:
        return np.concatenate([x, x, x], axis=2)
    if C == 4:
        color = x[:, :, 0:3].astype(np.float32)
        alpha = x[:, :, 3:4].astype(np.float32) / 255.0
        y = color * alpha + 255.0 * (1.0 - alpha)
        y = y.clip(0, 255).astype(np.uint8)
        return y
def resize_image(input_image, resolution):
    H, W, C = input_image.shape
    H = float(H)
    W = float(W)
    k = float(resolution) / min(H, W)
    H *= k
    W *= k
    H = int(np.round(H / 64.0)) * 64
    W = int(np.round(W / 64.0)) * 64
    img = cv2.resize(input_image, (W, H), interpolation=cv2.INTER_LANCZOS4 if k > 1 else cv2.INTER_AREA)
    return img

In [12]:
apply_openpose = OpenposeDetector()

In [48]:
input_image = np.array(Image.open("frame1_upscale.png"))
input_image = HWC3(input_image)
detected_map = apply_openpose(resize_image(input_image, detect_resolution),
                                 include_body=True,
                                 include_hand=True,
                                 include_face=True,
                                 use_dw_pose=False)
print(detected_map.shape)
image = Image.fromarray(detected_map) 
image.save("motion_frames/Bandit/chop/test.png")
detected_map = HWC3(detected_map)
img = resize_image(input_image, image_resolution)
H, W, C = img.shape

detected_map = cv2.resize(detected_map, (W, H), interpolation=cv2.INTER_NEAREST)

# control = torch.from_numpy(detected_map.copy())#.float().cuda() / 255.0

(512, 512, 3)


In [49]:
image = Image.fromarray(detected_map) 
image.save("test.png")